# Import modelflow

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from ipywidgets import interact,Dropdown,Checkbox
from IPython.display import display, clear_output,Latex, Markdown
import pandas as pd

import modeljupyter as mj
from modelvis import waterplot


from modelclass import model
model.widescreen()

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;
}

<IPython.core.display.Javascript object>

# Load model and data, and run 

In [3]:
if not 'masia' in locals():
    print('Load model')
    masia,baseline = model.modelload('Asia.pcim',run=1,silent=1)

Load model


In [4]:
from inject import inject
inject(masia)

In [5]:
masia.var_description = masia.defsub({k : v.replace('CO2','$CO^2$') for k,v in masia.var_description.items()})

In [ ]:
co2_input_var = [var for var in masia.totgraph.predecessors('WLD_CO2') if var.endswith('CO2')]

In [ ]:
co2_baseline = baseline.loc[:,co2_input_var].pipe(lambda df:df.rename(columns={c : masia.iso_dict.get(cname := c.split('_')[0],cname) for c in df.columns}))

# Variables to inputwidget

In [ ]:
outvar = 'GDI GCARBR CO2 LNN'.split()
displayvar = 'WLD_CO2 ASP_CO2 G20_CO2 '

#  masia.countries_GCARBR_A is list of all countries in the model with a countries_GCARBR_A 
#  masia.countries_ASEAN is list of all countries in ASEAN 
countries_rest = [country for country in masia.countries_GCARBR_A if country not in masia.countries_ASEAN]

slidedef_allasean =  {'All ASEAN countries' : {'var': ' '.join([f'{country}_GCARBR_A' for country in masia.countries_ASEAN ])
                          , 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10,'dec':0}}
slidedef_allrest =  {'Rest of the world' : {'var': 
    ' '.join([f'{country}_GCARBR_A' for country in countries_rest])
                          , 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10,'dec':0}}
slidedef_countries = {masia.iso_dict.get(country,country)+', Extra' : 
                      {'var': f'{country}_GCARBR_A', 'value': 0.0,'min':-40,'max':40,'op':'+impulse','step':10,'dec':0} 
                      for country in  masia.countries_ASEAN}

slidedef = {**slidedef_allasean,**slidedef_allrest,**slidedef_countries}

In [ ]:
masia['wld_co2'].rename().plot(colrow=1,title='Baseline $CO^2$ emmision',top=0.9,ysize=5);

# Emmision with no policy 
Countries where the emmision is below the threshold are all attributed to the "small" label

In [ ]:
@interact(Threshold = (0,3000,100))
def explain(Periode   = Dropdown(options = masia.current_per,value=2050),
            Threshold =700):

    waterplot(co2_baseline.loc[[Periode],:].T,autosum=1,threshold = Threshold,allsort=1,title = '$CO^2$ emmision in',top=0.88);

# Make some policy 

In [ ]:
inject(masia)
input = masia.inputwidget_asia(2021,2050,slidedef=slidedef
                   ,showout=True,varpat=displayvar,showvar=True,go_now=1)

# Where does the reduction originate
The threshold leved and year can be chosen.
Countries where the reduction is below the threshold are all attributed to the "small" label  

In [ ]:

@interact(Threshold = (0,300,10))
def explain(Periode   = Dropdown(options = masia.current_per,value=2050),
            Threshold =200):
    masia.dekomp_plot('WLD_CO2',pct=0,threshold=Threshold,nametrans=masia.nametrans)
    masia.dekomp_plot_per('WLD_CO2',pct=0,threshold=Threshold,per=Periode,sort=True,nametrans=masia.nametrans)


In [ ]:
print(masia.get_eq_des('USA_EC',show_all=1))

In [ ]:
masia.USA_EC.tracepre(HR=0)

# Inversion of the model

In [6]:
instruments = [[f'{country}_GCARBR_A' for country in masia.countries_GCARBR_A]]
instruments

[['AFG_GCARBR_A',
  'ARM_GCARBR_A',
  'AUS_GCARBR_A',
  'AZE_GCARBR_A',
  'BGD_GCARBR_A',
  'BTN_GCARBR_A',
  'BRN_GCARBR_A',
  'KHM_GCARBR_A',
  'CHN_GCARBR_A',
  'FJI_GCARBR_A',
  'GEO_GCARBR_A',
  'HKG_GCARBR_A',
  'IND_GCARBR_A',
  'IDN_GCARBR_A',
  'IRN_GCARBR_A',
  'JPN_GCARBR_A',
  'KAZ_GCARBR_A',
  'KIR_GCARBR_A',
  'KGZ_GCARBR_A',
  'LAO_GCARBR_A',
  'MAC_GCARBR_A',
  'MYS_GCARBR_A',
  'MDV_GCARBR_A',
  'FSM_GCARBR_A',
  'MNG_GCARBR_A',
  'MMR_GCARBR_A',
  'NPL_GCARBR_A',
  'NZL_GCARBR_A',
  'PAK_GCARBR_A',
  'PNG_GCARBR_A',
  'PHL_GCARBR_A',
  'KOR_GCARBR_A',
  'RUS_GCARBR_A',
  'WSM_GCARBR_A',
  'SGP_GCARBR_A',
  'SLB_GCARBR_A',
  'LKA_GCARBR_A',
  'TJK_GCARBR_A',
  'THA_GCARBR_A',
  'TLS_GCARBR_A',
  'TON_GCARBR_A',
  'TUR_GCARBR_A',
  'TKM_GCARBR_A',
  'UZB_GCARBR_A',
  'VUT_GCARBR_A',
  'VNM_GCARBR_A']]

In [32]:
target = pd.DataFrame([36_440],index=[2049],columns=['WLD_CO2'])
target

,WLD_CO2
2049,36440


In [36]:
delay = 2049-2021
delay 

28

In [37]:
from modelinvert import targets_instruments

In [38]:
t_i = targets_instruments(databank=baseline,targets=target,instruments=instruments,model=masia,DefaultImpuls=50.,defaultconv=20,varimpulse=True)
t_i.targets

,WLD_CO2
2049,36440


In [25]:
t_i.jacobi(2049,28)

,Instrument_0
WLD_CO2,5227.991739


In [40]:
t_i.debug=0
res = t_i(delay=29,varimpulse=True,silent=0)

Period: 2049
Period: 2049  Target instrument iteration: 0
Period: 2049  Target instrument iteration: 1
Period: 2049  Target instrument iteration: 2
Period: 2049  Target instrument iteration: 3
Period: 2049  Target instrument iteration: 4
Period: 2049  Target instrument iteration: 5
Period: 2049  Target instrument iteration: 6
Period: 2049  Target instrument iteration: 7
Period: 2049  Target instrument iteration: 8
Period: 2049  Target instrument iteration: 9
Period: 2049  Target instrument iteration: 10
Period: 2049  Target instrument iteration: 11
Period: 2049  Target instrument iteration: 12
Period: 2049  Target instrument iteration: 13
Period: 2049  Target instrument iteration: 14
Period: 2049  Target instrument iteration: 15
Period: 2049  Target instrument iteration: 16
Period: 2049  Target instrument iteration: 17
Period: 2049  Target instrument iteration: 18
Period: 2049  Target instrument iteration: 19


In [47]:
masia.calculate_freq_list(masia.coreorder)

[('*', 15973),
 ('+', 13676),
 ('-', 7220),
 ('/', 6587),
 ('LOG', 3943),
 ('EXP', 504),
 ('**', 202),
 ('Total', 48105)]

In [52]:
len(masia.epiorder)

1964

In [22]:
masia.wld_co2

Endogeneous: WLD_CO2: Territorial carbon dioxide emissions, Mt$CO^2$ ,World 
Formular: FRML <Z,EXO> WLD_CO2 =( +AFG_CO2+ARM_CO2+AUS_CO2+AZE_CO2+BGD_CO2+BTN_CO2+BRN_CO2+KHM_CO2+CHN_CO2+FJI_CO2+GEO_CO2+HKG_CO2+IND_CO2+IDN_CO2+IRN_CO2+JPN_CO2+KAZ_CO2+KIR_CO2+KGZ_CO2+LAO_CO2+MAC_CO2+MYS_CO2+MDV_CO2+FSM_CO2+MNG_CO2+MMR_CO2+NPL_CO2+NZL_CO2+PAK_CO2+PNG_CO2+PHL_CO2+KOR_CO2+RUS_CO2+WSM_CO2+SGP_CO2+SLB_CO2+LKA_CO2+TJK_CO2+THA_CO2+TLS_CO2+TON_CO2+TUR_CO2+TKM_CO2+UZB_CO2+VUT_CO2+VNM_CO2+USA_CO2+DEU_CO2+FRA_CO2+GBR_CO2+ITA_CO2+NLD_CO2+IRL_CO2+CHE_CO2+TWN_CO2+AFR_CO2+EUR_CO2+LAT_CO2+MNA_CO2 + WLD_CO2_A )*(1-WLD_CO2_D)+WLD_CO2_X*WLD_CO2_D$

WLD_CO2  : Territorial carbon dioxide emissions, Mt$CO^2$ ,World
AFG_CO2  : Territorial carbon dioxide emissions, Mt$CO^2$ ,Afghanistan
AFR_CO2  : Territorial carbon dioxide emissions, Mt$CO^2$ ,SubSaharanAfrica
ARM_CO2  : Territorial carbon dioxide emissions, Mt$CO^2$ ,Armenia
AUS_CO2  : Territorial carbon dioxide emissions, Mt$CO^2$ ,Australia
AZE_CO2  : Territo

In [30]:
masia.exodif()

,AFG_GCARBR_A,ARM_GCARBR_A,AUS_GCARBR_A,AZE_GCARBR_A,BGD_GCARBR_A,BRN_GCARBR_A,BTN_GCARBR_A,CHN_GCARBR_A,FJI_GCARBR_A,FSM_GCARBR_A,...,THA_GCARBR_A,TJK_GCARBR_A,TKM_GCARBR_A,TLS_GCARBR_A,TON_GCARBR_A,TUR_GCARBR_A,UZB_GCARBR_A,VNM_GCARBR_A,VUT_GCARBR_A,WSM_GCARBR_A
2020,57.718537,57.718537,57.718537,57.718537,57.718537,57.718537,57.718537,57.718537,57.718537,57.718537,...,57.718537,57.718537,57.718537,57.718537,57.718537,57.718537,57.718537,57.718537,57.718537,57.718537
